In [43]:
from functools import partial
import mysql.connector
from queue import SimpleQueue as Queue
import urllib.request
from bs4 import BeautifulSoup as bs

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        # print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)

class Page():
    def __init__(self, url, title, content, sub_urls):
        self.url: str = url
        self.title: str = title
        self.content: str = content
        self.sub_urls: list = sub_urls

    def __str__(self):
        return f"Page: '{self.title}'\n\nContent: \n{self.content}"

def read_data_from_url(url: str):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    return response.read() # The data u need

def extract_links(page_data: str):
    links = bs(page_data, 'html.parser')
    return set(map(lambda x: x.get('href'),links.find_all("a")))

def extract_text_and_title(page_data: str):
    soup = bs(page_data, 'html.parser')
    return soup.title.get_text(), ' '.join(list(map(lambda x: x.get_text(), soup.find_all("p")))[1:])

def build_page(url: str) -> Page:
    data = read_data_from_url(url)
    title, content = extract_text_and_title(data)
    sub_urls = extract_links(data)
    return Page(url, title, content, sub_urls)

class Crawler():

    def __init__(self, db, url):
        self.db = db
        self.root_url = url
        self.pagelist = []
        self.finished_url_set = set()
        self.queue = Queue()
        self.queue.put(self.root_url)
        self.finished_url_set.add(self.root_url)

    def crawl_step(self, url):
        try:
            page = build_page(url)
        except Exception:
            # print(f"Could not interpret page at url {url}, skipping...")
            return
        self.pagelist.append(page)
        self.finished_url_set.add(url)
        for p in page.sub_urls:
            if p not in self.finished_url_set:
                self.queue.put(p)
        # print(f"Process complete for url {url}")
        return

    def crawl(self, quant: int):
        for i in range(quant):
            if self.queue.empty():
                return
            self.crawl_step(self.queue.get())
            if (i==quant-1) or (i%25==0 and i>0): # Periodic savepoints
                print(f"Progress Report: {i}/{quant} = {100*i/quant:.02f}%")
                for p in self.pagelist:
                    try:
                        self.db(f'INSERT INTO Pages (url, title, content) VALUES ("{p.url}", "{p.title}", "{p.content}")')
                        self.db("COMMIT")
                    except Exception as e:
                        # print(f"\nCould not commit to database!\n{e}\n")
                        continue
                self.pagelist.clear()

In [6]:
# db("DESCRIBE Pages")

In [44]:
# print(build_page(url))
# =====================================================================

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='nlp',
    database='nlp',
)

db = partial(run_db_query, connection)
url = "https://stackoverflow.com/questions/2081586/web-scraping-with-python"
crawler = Crawler(db,url)
# db("DELETE FROM Pages")
crawler.crawl(10000)
db("SELECT * FROM Pages LIMIT 5")
connection.close()


Progress Report: 0/10000 = 0.00%
Executando query:
Executando query:
Progress Report: 25/10000 = 0.25%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'http://www.google.com/") instead of just urlopen("http://www.google.com/"). For ' at line 1

Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'but what if" questions be encouraged or discouraged? - Parenting Stack Exchange"' at line 1

Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress Report: 300/10000 = 3.00%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'accepted". Accepting doesn't mean it's the best answer, it just means that it wo' at line 7

Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Progress Report: 325/10000 = 3.25%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executand

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress Report: 475/10000 = 4.75%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'What are the key aspects and contents of a good coding standards document?" #sta' at line 1

Progress Report: 500/10000 = 5.00%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax

...

Progress Report: 7775/10000 = 77.75%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'dribbled a lot and wasn’t allowed anything sharp" in "Good Omens"? - Literatur' at line 1

Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Dr Smith and an alien" and also "Dr Smith and a monster" but I keep getting led ' at line 1

Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress Report: 7800/10000 = 78.00%
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'alphabetic characters plus underscore (_)". It's fairly easy to remember that in' at line 1

Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'our" project, and it will be handed in as such, but it will be painfully obvious' at line 1

Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'embed" the observer in the model. It is the case, for example, of relativity (bo' at line 1

Executando query:

Could not commit to database!
1064 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress Report: 7975/10000 = 79.75%
Executando query:
Executando query:
Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'sticking the landing" means?  'Stick the landing' is from gymnastics, when peopl' at line 1

Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Flyvanse"  or "Tryvanse".  Ad campaigns and persuasive drug reps work on the hon' at line 2

Executando query:
Executando query:
Executando query:

Could not commit to database!
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'controlled by all players" so everyone 